## Homework: Search Evaluation

In this homework, we will evaluate the results of vector
search.

> It's possible that your answers won't match exactly. If it's the case, select the closest one.


## Required libraries

We will use minsearch and Qdrant. Make sure you have the most up-to-date versions:

```bash
pip install -U minsearch qdrant_client
``` 

minsearch should be at least 0.0.4.



## Evaluation data

For this homework, we will use the same dataset we generated
in the videos.

Let's get them:

```python
import requests
import pandas as pd

url_prefix = 'https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/03-evaluation/'
docs_url = url_prefix + 'search_evaluation/documents-with-ids.json'
documents = requests.get(docs_url).json()

ground_truth_url = url_prefix + 'search_evaluation/ground-truth-data.csv'
df_ground_truth = pd.read_csv(ground_truth_url)
ground_truth = df_ground_truth.to_dict(orient='records')
```

Here, `documents` contains the documents from the FAQ database
with unique IDs, and `ground_truth` contains generated
question-answer pairs. 

Also, we will need the code for evaluating retrieval:

```python
from tqdm.auto import tqdm

def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }
```

In [4]:
import requests
import pandas as pd

url_prefix = 'https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/03-evaluation/'
docs_url = url_prefix + 'search_evaluation/documents-with-ids.json'
documents = requests.get(docs_url).json()

ground_truth_url = url_prefix + 'search_evaluation/ground-truth-data.csv'
df_ground_truth = pd.read_csv(ground_truth_url)
ground_truth = df_ground_truth.to_dict(orient='records')

In [5]:
from tqdm.auto import tqdm

def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

## Q1. Minsearch text

Now let's evaluate our usual minsearch approach, indexing documents with:

```python
text_fields=["question", "section", "text"],
keyword_fields=["course", "id"]
```

but tweak the parameters for search. Let's use the following boosting params:

```python
boost = {'question': 1.5, 'section': 0.1}
```

What's the hitrate for this approach?

0.64

0.74

0.84

0.94


In [7]:
import minsearch

index = minsearch.Index(
    text_fields=["question", "section", "text"],
    keyword_fields=["course", "id"]
)

index.fit(documents)

In [8]:
def minsearch_search(query: str, course: str):
    boost = {'question': 1.5, 'section': 0.1}

    results = index.search(
        query=query,
        filter_dict={ 'course': course },
        boost_dict=boost,
        num_results=5
    )

    return results

In [9]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    results = minsearch_search(query=q['question'], course=q['course'])
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

100%|██████████| 4627/4627 [00:07<00:00, 611.24it/s]


In [11]:
print(f"Hit rate: {hit_rate(relevance_total=relevance_total)}, MRR: {mrr(relevance_total=relevance_total)}")

Hit rate: 0.848714069591528, MRR: 0.7288235717887772


## Embeddings 

The latest version of minsearch also supports vector search. 
We will use it:

```python
from minsearch import VectorSearch
```

We will also use TF-IDF and Singular Value Decomposition to 
create embeddings from texts. You can refer to our
["Create Your Own Search Engine" workshop](https://github.com/alexeygrigorev/build-your-own-search-engine)
if you want to know more about it.

```python
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
```

Let's create embeddings for the "question" field:

```python
texts = []

for doc in documents:
    t = doc['question']
    texts.append(t)

pipeline = make_pipeline(
    TfidfVectorizer(min_df=3),
    TruncatedSVD(n_components=128, random_state=1)
)
X = pipeline.fit_transform(texts)
```

## Q2. Vector search for question

Now let's index these embeddings with minsearch:

```python
vindex = VectorSearch(keyword_fields={'course'})
vindex.fit(X, documents)
```

Evaluate this seach method. What's MRR for it?

- 0.25
- 0.35
- 0.45
- 0.55

In [12]:
from minsearch import VectorSearch
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline


In [13]:
texts = []

for doc in documents:
    t = doc['question']
    texts.append(t)

pipeline = make_pipeline(
    TfidfVectorizer(min_df=3),
    TruncatedSVD(n_components=128, random_state=1)
)
X = pipeline.fit_transform(texts)

In [16]:
vindex = minsearch.VectorSearch(keyword_fields={'course'})
vindex.fit(X, documents)

In [17]:
relevance_total_2 = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    query_vector = pipeline.transform([q['question']])
    filter_dict = {'course': q['course']}
    results = vindex.search(query_vector=query_vector,filter_dict=filter_dict,num_results=5)
    relevance = [d['id'] == doc_id for d in results]
    relevance_total_2.append(relevance)

100%|██████████| 4627/4627 [00:04<00:00, 1127.99it/s]


In [20]:
mrr_2 = mrr(relevance_total=relevance_total_2)
print(f"MRR: {mrr_2}")

MRR: 0.35720048987825087


## Q3. Vector search for question and answer

We only used question in Q2. We can use both question and answer:

```python
texts = []

for doc in documents:
    t = doc['question'] + ' ' + doc['text']
    texts.append(t)
```

Using the same pipeline (min_df=3 for TF-IDF vectorizer and n_components=128` for SVD), evaluate the performance of this approach

What's the hitrate?
- 0.62
- 0.72
- 0.82
- 0.92

In [21]:
texts = []

for doc in documents:
    t = doc['question'] + ' ' + doc['text']
    texts.append(t)

In [26]:
X = pipeline.fit_transform(texts)

vindex = VectorSearch(keyword_fields={'course'})
vindex.fit(X, documents)

In [27]:
relevance_total_3 = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    query_vector = pipeline.transform([q['question']])
    filter_dict = {'course': q['course']}
    results = vindex.search(query_vector=query_vector,filter_dict=filter_dict,num_results=5)
    relevance = [d['id'] == doc_id for d in results]
    relevance_total_3.append(relevance)

100%|██████████| 4627/4627 [00:04<00:00, 974.12it/s] 


In [30]:
hit_rate_3 = hit_rate(relevance_total_3)
print(f"Hit rate: {hit_rate_3}")

Hit rate: 0.8210503566025502


## Q4. Qdrant

Now let's evaluate the following settings in Qdrant:

- text = doc['question'] + ' ' + doc['text']
- model_handle = "jinaai/jina-embeddings-v2-small-en"
- limit = 5

What's the MRR?
- 0.65
- 0.75
- 0.85
- 0.95


In [43]:
from qdrant_client import QdrantClient, models

client = QdrantClient("http://localhost:6333")

EMBEDDING_DIMENSIONALITY = 512
EMBEDDING_MODEL = "jinaai/jina-embeddings-v2-small-en"
COLLECTION_NAME = "zoomcamp-eval"
LIMIT = 5



In [35]:
client.create_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)

# client.delete_collection(collection_name=COLLECTION_NAME)

True

In [36]:
points = []
id = 0

for doc in documents:

    point = models.PointStruct(
        id=id,
        vector=models.Document(text=doc['text'], model=EMBEDDING_MODEL),
        payload={
            "text": doc['question'] + ' ' + doc['text'],
            "section": doc['section'],
            "course": doc['course'],
            "uuid": doc['id']
        }
    )
    points.append(point)

    id += 1

In [38]:
# upsert pts
client.upsert(points=points, collection_name=COLLECTION_NAME)

Fetching 5 files: 100%|██████████| 5/5 [00:01<00:00,  3.58it/s]


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [41]:
client.create_payload_index(
    collection_name=COLLECTION_NAME,
    field_name="course",
    field_schema="keyword"
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [42]:
def search_in_course (query: str, course='mlops-zoomcamp', limit=1):
    results = client.query_points(
        collection_name=COLLECTION_NAME,
        query=models.Document(  # embed the query text locally with "jinaai/jina-embeddings-v2-small-en"
            text=query,
            model=EMBEDDING_MODEL
        ),
        query_filter=models.Filter( # filter by course name
            must= [
                models.FieldCondition(
                    key='course',
                    match=models.MatchValue(value=course)
                )
            ]
        ),
        limit=limit, # top closest matches
        with_payload=True # to get metadata in the results
    )
    return results

In [44]:
# calculate relevance
relevance_total_4 = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    results = search_in_course(
        query=q['question'],
        course=q['course'],
        limit=LIMIT
        )
    relevance = [d.payload['uuid'] == doc_id for d in results.points]
    relevance_total_4.append(relevance)

100%|██████████| 4627/4627 [00:38<00:00, 118.91it/s]


In [45]:
mrr_4 = mrr(relevance_total=relevance_total_4)

print(f"MRR: {mrr_4}")

MRR: 0.7331028023917593


## Q5. Cosine simiarity

In the second part of the module, we looked at evaluating
the entire RAG approach. In particular, we looked at 
comparing the answer generated by our system with the actual
answer from the FAQ.

One of the ways of doing it is using the cosine similarity. 
Let's see how to calculate it.

Cosine similarity is a dot product between two normalized vectors.
In geometrical sense, it's the cosine of the angle between
the vectors. Look up "cosine similarity geometry" if you want to
learn more about it.

For us, it means that we need two things:

- First, we normalize each of the vectors
- Then, compute the dot product

So, we get this:

```python
def cosine(u, v):
    u = normalize(u)
    v = normalize(v)
    return u.dot(v)
```

For normalization, we first compute the vector norm (its length),
and then divide the vector by it:

```python
def normalize(u):
    norm = np.sqrt(u.dot(u))
    return u / norm
```

(where `np` is `import numpy as np`)

Or we can simplify it:

```python
def cosine(u, v):
    u_norm = np.sqrt(u.dot(u))
    v_norm = np.sqrt(v.dot(v))
    return u.dot(v) / (u_norm * v_norm)
```

Now let's use this function to compute the
A->Q->A cosine similarity.

We will use the results from [our gpt-4o-mini evaluations](https://github.com/DataTalksClub/llm-zoomcamp/blob/main/03-evaluation/rag_evaluation/data/results-gpt4o-mini.csv):


```python
results_url = url_prefix + 'rag_evaluation/data/results-gpt4o-mini.csv'
df_results = pd.read_csv(results_url)
```


When creating embeddings, we will use a simple way -
the same we used in the [Embeddings](#embeddings) section:

```python
pipeline = make_pipeline(
    TfidfVectorizer(min_df=3),
    TruncatedSVD(n_components=128, random_state=1)
)
```

Let's fit the vectorizer on all the text data we have:

```python
pipeline.fit(df_results.answer_llm + ' ' + df_results.answer_orig + ' ' + df_results.question)
```

Now use the `transform` methon of the pipeline to create the embeddings and calculate the cosine similarity between each
pair.

What's the average cosine?

- 0.64
- 0.74
- 0.84
- 0.94

This is how you do it:

- For each answer pair, compute
    - `v_llm` for the answer from the LLM 
    - `v_orig` for the original answer
    - then compute the cosine between them
- At the end, take the average


In [51]:
import numpy as np
import os

In [58]:
def cosine(u, v):
    u_norm = np.sqrt(u.dot(u))
    v_norm = np.sqrt(v.dot(v))
    return u.dot(v) / (u_norm * v_norm)

In [59]:
current_path = os.getcwd()
results_path = os.path.join(current_path, 'rag_evaluation', 'data', 'results-gpt4o-mini.csv')
df_results = pd.read_csv(results_path)

In [60]:
pipeline = make_pipeline(
    TfidfVectorizer(min_df=3),
    TruncatedSVD(n_components=128, random_state=1)
)

In [61]:
pipeline.fit(df_results.answer_llm + ' ' + df_results.answer_orig + ' ' + df_results.question)

,steps,"[('tfidfvectorizer', ...), ('truncatedsvd', ...)]"
,transform_input,None
,memory,None
,verbose,False
,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,None
,tokenizer,None


In [63]:
v_llm = pipeline.transform(df_results.answer_llm)
v_orig = pipeline.transform(df_results.answer_orig)

In [65]:
def cos_sim(A, B):
    A_norm = A / np.linalg.norm(A, axis=1, keepdims=True)
    B_norm = B / np.linalg.norm(B, axis=1, keepdims=True)
    return np.sum(A_norm * B_norm, axis=1)

In [69]:
cosine_similarity = cos_sim(v_orig, v_llm)
cosine_similarity_avg = np.mean(cosine_similarity)
print(f"Avg cosine similarity: {cosine_similarity_avg}")

Avg cosine similarity: 0.8415841233490402


## Q6. Rouge

And alternative way to see how two texts are similar is ROUGE.

This is a set of metrics that compares two answers based on the overlap of n-grams, word sequences, and word pairs.

It can give a more nuanced view of text similarity than just cosine similarity alone.

We don't need to implement it ourselves, there's a python package for it:

```sh
pip install rouge
```

(The latest version at the moment of writing is 1.0.1)

Let's compute the ROUGE score between the answers at the index 10 of our dataframe (doc_id=5170565b)

```python
from rouge import Rouge
rouge_scorer = Rouge()

r = df_results.iloc[10]
scores = rouge_scorer.get_scores(r.answer_llm, r.answer_orig)[0]
scores
```

There are three scores: rouge-1, rouge-2 and rouge-l, and precision, recall and F1 score for each.

- `rouge-1` - the overlap of unigrams,
- `rouge-2` - bigrams,
- `rouge-l` - the longest common subsequence

For the 10th document, Rouge-1 F1 score is 0.45

Let's compute it for the pairs in the entire dataframe. What's the average Rouge-1 F1?
- 0.25
- 0.35
- 0.45
- 0.55


In [70]:
from rouge import Rouge
rouge_scorer = Rouge()

r = df_results.iloc[10]
scores = rouge_scorer.get_scores(r.answer_llm, r.answer_orig)[0]
scores

{'rouge-1': {'r': 0.45454545454545453,
  'p': 0.45454545454545453,
  'f': 0.45454544954545456},
 'rouge-2': {'r': 0.21621621621621623,
  'p': 0.21621621621621623,
  'f': 0.21621621121621637},
 'rouge-l': {'r': 0.3939393939393939,
  'p': 0.3939393939393939,
  'f': 0.393939388939394}}

In [71]:
def get_rouge1_f1(r):
    try:
        return rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]['rouge-1']['f']
    except Exception:
        return None

In [76]:
df_results['rouge1_f1'] = df_results.apply(get_rouge1_f1, axis=1)
avg_rouge1_f1 = df_results['rouge1_f1'].mean()
print(f"Avg ROUGE-1 F1: {avg_rouge1_f1}")

Avg ROUGE-1 F1: 0.3516946452113943
